In [49]:
import xgboost as xgb
import tqdm as tq
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
from tqdm import tqdm
from xgboost import XGBRegressor
import numpy as np
import warnings
import copy

In [24]:
warnings.filterwarnings('ignore')

### Train Data Preprocessing

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.pop('일조(hr)')
train.pop('일사(MJ/m2)')

train['일시'] = pd.to_datetime(train['일시'], format='%Y%m%d %H')
train['month'] = train['일시'].dt.month
train['day'] = train['일시'].dt.weekday
train['hour'] = train['일시'].dt.hour
train['holiday'] = train.apply(lambda x: 0 if x['day']<5 else 1, axis=1)
train['sin_time'] = np.sin(2*np.pi*train.hour/24) # cyclical encoding
train['cos_time'] = np.cos(2*np.pi*train.hour/24)
train['DI'] = 1.8*train['기온(C)'] - 0.55*(1-train['습도(%)'])*(1.8*train['기온(C)']-26) + 32 # 불쾌지수

train.pop('num_date_time')
train.pop('일시')
train.pop('hour')
train['power'] = train['전력소비량(kWh)']
train.pop('전력소비량(kWh)')
train['강수량(mm)'] = train['강수량(mm)'].fillna(0)
train['풍속(m/s)'] = train['풍속(m/s)'].fillna(0)
train['습도(%)'] = train['습도(%)'].fillna(0)

In [4]:
test = pd.read_csv('test.csv')

In [5]:
test['일시'] = pd.to_datetime(test['일시'], format='%Y%m%d %H')
test['month'] = test['일시'].dt.month
test['day'] = test['일시'].dt.weekday
test['hour'] = test['일시'].dt.hour
test['holiday'] = test.apply(lambda x: 0 if x['day']<5 else 1, axis=1)
test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)
test['DI'] = 1.8*test['기온(C)'] - 0.55*(1-test['습도(%)'])*(1.8*test['기온(C)']-26) + 32

test.pop('num_date_time')
test.pop('일시')
test.pop('hour')

0         0
1         1
2         2
3         3
4         4
         ..
16795    19
16796    20
16797    21
16798    22
16799    23
Name: hour, Length: 16800, dtype: int64

### Training

In [6]:
import os
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split

In [7]:
def SMAPE(true, pred):
    return np.mean((np.abs(true - pred))/(np.abs(true) + np.abs(pred))) * 100    

In [8]:
def tr_ts_split(train, test, number, val_hour):
    x = train.loc[train['건물번호']==number].drop(['건물번호','power'], axis=1)
    y = train.loc[train['건물번호']==number, 'power']

    if val_hour == 0:
        return x, None, y, None
    else:
        x_train, x_valid, y_train, y_valid = x[:-val_hour], x[-val_hour:], y[:-val_hour], y[-val_hour:]
        return x_train, x_valid.reset_index(drop=True), y_train, y_valid.reset_index(drop=True)

In [9]:
def fit_and_pred_XGB(train, test, number, seed=0):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, number, 7*24)

    xgb_reg = XGBRegressor(n_estimators=10000, eta=0.01, seed=seed,
                            gpu_id=0, tree_method='gpu_hist', predictor='gpu_predictor')
    xgb_reg.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)],
                early_stopping_rounds=300, verbose=False)
    pred = xgb_reg.predict(x_valid)
    pred = pd.Series(pred)
    
    x_test = test[test['건물번호']==number]
    y_test_pred = xgb_reg.predict(x_test.drop('건물번호', axis=1))

    smape_val = SMAPE(y_valid, pred)
    r2_val = sklearn.metrics.r2_score(y_valid, pred)
    print(f'SMAPE: {smape_val}')
    print(f'R2: {r2_val}')
    return smape_val, r2_val, y_valid, pred, y_test_pred

### Improvement

In [10]:
train['적정온도차이'] = train['기온(C)'] - 26

In [11]:
test['적정온도차이'] = test['기온(C)'] - 26

In [12]:
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

In [36]:
def fit_and_pred_XGB_with_weight(train, test, number, weight, seed=0):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, number, 7*24)

    xgb_reg = XGBRegressor(n_estimators=10000, eta=0.01, seed=seed,
                            gpu_id=0, tree_method='gpu_hist', predictor='gpu_predictor')
    xgb_reg.set_params(**{'objective':weighted_mse(weight)})
    xgb_reg.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)],
                early_stopping_rounds=300, verbose=False)
    pred = xgb_reg.predict(x_valid)
    pred = pd.Series(pred)
    
    x_test = test[test['건물번호']==number]
    y_test_pred = xgb_reg.predict(x_test.drop('건물번호', axis=1))

    smape_val = SMAPE(y_valid, pred)
    r2_val = sklearn.metrics.r2_score(y_valid, pred)
    print(f'SMAPE: {smape_val}')
    print(f'R2: {r2_val}')
    return smape_val, r2_val, y_valid, pred, y_test_pred

In [203]:
pred_val_ens= []
test_ens = []

summary_list = []
ans_val_list = []
pred_val_list = []
pred_test_list = []

for i in tqdm(range(100)):
    print(f"===== < BUILDING NO.{i+1} > =====")
    summary_dict = {}

    smape_val, r2_val, ans_val, pred_val, pred_test = fit_and_pred_XGB_with_weight(train, test, i+1, 100, seed=0)

    summary_dict['건물번호'] = i+1
    summary_dict['SMAPE'] = smape_val
    summary_dict['R2'] = r2_val
    summary_list.append(summary_dict)
    ans_val_list.append(ans_val)
    pred_val_list.append(pred_val)
    pred_test_list.append(pred_test)

ans_val_whole = np.concatenate(ans_val_list)
pred_val_whole = np.concatenate(pred_val_list)
pred_test_whole = np.concatenate(pred_test_list)
smape_val =SMAPE(ans_val_whole, pred_val_whole)

  0%|          | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


===== < BUILDING NO.1 > =====


  1%|          | 1/100 [00:01<03:12,  1.94s/it]

SMAPE: 3.5419388838477404
R2: 0.9563832166343148
===== < BUILDING NO.2 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
  2%|▏         | 2/100 [00:03<02:52,  1.76s/it]

...

SMAPE: 7.2588277284224985
R2: 0.8560677105509416
===== < BUILDING NO.99 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
 99%|█████████▉| 99/100 [03:48<00:02,  2.10s/it]

SMAPE: 1.5314447971624443
R2: 0.9820596675396264
===== < BUILDING NO.100 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
100%|██████████| 100/100 [03:50<00:00,  2.31s/it]

SMAPE: 2.597097674198934
R2: 0.967838655132623


In [204]:
summary_list

[{'건물번호': 1, 'SMAPE': 3.5419388838477404, 'R2': 0.9563832166343148},
 {'건물번호': 2, 'SMAPE': 4.295780152455329, 'R2': 0.8651174662607124},
 {'건물번호': 3, 'SMAPE': 4.976127431146375, 'R2': 0.8282886548210623},
 {'건물번호': 4, 'SMAPE': 1.916974758959043, 'R2': 0.9749491993399805},
 {'건물번호': 5, 'SMAPE': 2.8188883844313564, 'R2': 0.972446435735248},
 {'건물번호': 6, 'SMAPE': 1.8247299710309055, 'R2': 0.9802984729616211},
 {'건물번호': 7, 'SMAPE': 4.181533402777831, 'R2': 0.906970899840581},
 {'건물번호': 8, 'SMAPE': 2.8739138223687473, 'R2': 0.6702282399188223},
 {'건물번호': 9, 'SMAPE': 1.8549493161639723, 'R2': 0.984736771457575},
 {'건물번호': 10, 'SMAPE': 3.5416225095446876, 'R2': 0.4322418016566194},
 {'건물번호': 11, 'SMAPE': 1.686647477272509, 'R2': 0.8955227858960689},
 {'건물번호': 12, 'SMAPE': 1.9311341180054387, 'R2': 0.8995117473179703},
 {'건물번호': 13, 'SMAPE': 3.004711373509035, 'R2': 0.3961679661426334},
 {'건물번호': 14, 'SMAPE': 7.909728524427743, 'R2': 0.34326299780211256},
 {'건물번호': 15, 'SMAPE': 1.8971932943063

In [205]:
smape_val

2.790566542032983

In [93]:
sub['answer'] = pred_test_whole
sub.to_csv('xgb_sub_16.csv', index=False)

### Parameter Tuning - Grid Search CV

In [55]:
from sklearn.model_selection import GridSearchCV, PredefinedSplit

In [58]:
# 최종적으로 최고의 모델 뽑는 부분
pred_val_ens= []
test_ens = []

summary_list = []
ans_val_list = []
pred_val_list = []
pred_test_list = []
feature_list = []
for i in tqdm(range(100)):
    print(f"===== < BUILDING NO.{i+1} > =====")
    summary_dict = {}

    smape_val, r2_val, ans_val, pred_val, pred_test, features = fit_and_pred_XGB_with_importance_fs(train, test, i+1, seed=0)

    summary_dict['건물번호'] = i+1
    summary_dict['SMAPE'] = smape_val
    summary_dict['R2'] = r2_val
    summary_list.append(summary_dict)
    ans_val_list.append(ans_val)
    pred_val_list.append(pred_val)
    pred_test_list.append(pred_test)
    feature_list.append(copy.deepcopy(features))

ans_val_whole = np.concatenate(ans_val_list)
pred_val_whole = np.concatenate(pred_val_list)
pred_test_whole = np.concatenate(pred_test_list)
smape_val =SMAPE(ans_val_whole, pred_val_whole)

  0%|          | 0/100 [00:00<?, ?it/s]

===== < BUILDING NO.1 > =====


  1%|          | 1/100 [00:26<43:44, 26.51s/it]

SMAPE: 3.7324068823625947
R2: 0.9424291261073321
===== < BUILDING NO.2 > =====


  2%|▏         | 2/100 [00:41<32:16, 19.76s/it]

...

SMAPE: 6.6799015201695715
R2: 0.8752175087620719
===== < BUILDING NO.99 > =====


 99%|█████████▉| 99/100 [34:33<00:19, 19.77s/it]

SMAPE: 1.6683536388197868
R2: 0.9795981191390115
===== < BUILDING NO.100 > =====


100%|██████████| 100/100 [34:53<00:00, 20.93s/it]

SMAPE: 2.375489927434594
R2: 0.9732887823743216


In [69]:
df = pd.DataFrame(columns = ['n_estimators', 'eta', 'min_child_weight', 'max_depth', 'colsample_bytree', 'subsample'])
preds = np.array([])
grid = {
    'n_estimators':     [30, 50, 70, 100],
    'eta':              [0.01],
    'min_child_weight': np.arange(1, 8, 1),
    'max_depth':        np.arange(3, 9, 1),
    'colsample_bytree': np.arange(0.8, 1.0, 0.1),
    'subsample':        np.arange(0.8, 1.0, 0.1)
}

for i in tqdm(range(100)):
    print(f"===== < BUILDING NO.{i+1} > =====")
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, i+1, 7*24)
    gcv = GridSearchCV(estimator=XGBRegressor(seed=0, gpu_id=0,
                                              tree_method='gpu_hist', predictor='gpu_predictor'),
                       param_grid=grid,
                       scoring=make_scorer(SMAPE, greater_is_better=False),
                       cv=5,
                       refit=True,
                       verbose=True)
    gcv.fit(x_train[feature_list[i]], y_train)
    best = gcv.best_estimator_
    params = gcv.best_params_
    print(params)
    pred = best.predict(x_valid[feature_list[i]])
    print(f'SMAPE: {SMAPE(y_valid, pred)}')
    preds = np.append(preds, pred)
    df = pd.concat([df, pd.DataFrame(params, index=[0])], axis=0)

  0%|          | 0/100 [00:00<?, ?it/s]

===== < BUILDING NO.1 > =====
Fitting 5 folds for each of 672 candidates, totalling 3360 fits


  1%|          | 1/100 [04:14<6:59:52, 254.47s/it]

{'colsample_bytree': 0.8, 'eta': 0.01, 'max_depth': 3, 'min_child_weight': 6, 'n_estimators': 100, 'subsample': 0.9}
SMAPE: 23.72451880394092
===== < BUILDING NO.2 > =====
Fitting 5 folds for each of 672 candidates, totalling 3360 fits


  2%|▏         | 2/100 [08:28<6:55:08, 254.16s/it]

...

{'colsample_bytree': 0.8, 'eta': 0.01, 'max_depth': 3, 'min_child_weight': 7, 'n_estimators': 100, 'subsample': 0.9}
SMAPE: 20.102150373663253
===== < BUILDING NO.99 > =====
Fitting 5 folds for each of 672 candidates, totalling 3360 fits


 99%|█████████▉| 99/100 [5:42:37<03:40, 220.26s/it]

{'colsample_bytree': 0.8, 'eta': 0.01, 'max_depth': 3, 'min_child_weight': 7, 'n_estimators': 100, 'subsample': 0.9}
SMAPE: 21.232095907425162
===== < BUILDING NO.100 > =====
Fitting 5 folds for each of 672 candidates, totalling 3360 fits


100%|██████████| 100/100 [5:46:09<00:00, 207.70s/it]

{'colsample_bytree': 0.8, 'eta': 0.01, 'max_depth': 3, 'min_child_weight': 7, 'n_estimators': 100, 'subsample': 0.9}
SMAPE: 23.98998641295904


In [133]:
df.to_csv('hyperparameter_xgb.csv', index=False)

In [70]:
xgb_params = df

In [72]:
scores = []
best_iter = []

for i in tqdm(range(100)):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, i+1, 7*24)
    xgb_reg = XGBRegressor(n_estimators=10000, eta=0.01, min_child_weight=xgb_params.iloc[i, 2],
                            max_depth=xgb_params.iloc[i, 3], colsample_bytree=xgb_params.iloc[i, 4],
                            subsample=xgb_params.iloc[i, 5], seed=0)
    xgb_reg.set_params(**{'objective':weighted_mse(100)})
    xgb_reg.set_params(early_stopping_rounds=300)
    xgb_reg.fit(x_train[feature_list[i]], y_train, eval_set=[(x_train[feature_list[i]], y_train), (x_valid[feature_list[i]], y_valid)], verbose=False)
    y_pred = xgb_reg.predict(x_valid[feature_list[i]])
    pred = pd.Series(y_pred)
    sm = SMAPE(y_valid, y_pred)
    scores.append(sm)
    best_iter.append(xgb_reg.best_iteration) # 실제 best iteration은 이 값에 +1

100%|██████████| 100/100 [02:00<00:00,  1.20s/it]


In [75]:
alpha_list = []
smape_list = []

for i in tqdm(range(100)):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, i+1, 7*24)
    xgb = XGBRegressor(seed=0,
                        n_estimators=best_iter[i], eta=0.01, min_child_weight=xgb_params.iloc[i, 2],
                        max_depth=xgb_params.iloc[i, 3], colsample_bytree=xgb_params.iloc[i, 4], subsample=xgb_params.iloc[i, 5])
    xgb.fit(x_train[feature_list[i]], y_train)
    pred0 = xgb.predict(x_valid[feature_list[i]])
    best_alpha = 0
    score0 = SMAPE(y_valid, pred0)

    for j in [1, 2, 5, 7, 10, 25, 50, 75, 100]:
        xgb = XGBRegressor(seed=0,
                        n_estimators=best_iter[i], eta=0.01, min_child_weight=xgb_params.iloc[i, 2],
                        max_depth=xgb_params.iloc[i, 3], colsample_bytree=xgb_params.iloc[i, 4], subsample=xgb_params.iloc[i, 5])
        xgb.set_params(**{'objective': weighted_mse(j)})
        xgb.fit(x_train[feature_list[i]], y_train)
        pred1 = xgb.predict(x_valid[feature_list[i]])
        score1 = SMAPE(y_valid, pred1)
        if score1 < score0:
            best_alpha = j
            score0 = score1

    alpha_list.append(best_alpha)
    smape_list.append(score0)
    print(f"building {i+1} || best score: {score0} || alpha: {best_alpha}")

  1%|          | 1/100 [00:04<06:43,  4.07s/it]

building 1 || best score: 4.637872323582045 || alpha: 2


  2%|▏         | 2/100 [00:07<05:33,  3.40s/it]

building 2 || best score: 5.553603341035989 || alpha: 2


  3%|▎         | 3/100 [00:08<04:22,  2.71s/it]

...

building 98 || best score: 6.165975758298506 || alpha: 0


 99%|█████████▉| 99/100 [10:33<00:04,  4.19s/it]

building 99 || best score: 2.2969392107288638 || alpha: 1


100%|██████████| 100/100 [10:37<00:00,  6.37s/it]

building 100 || best score: 2.4989291948901022 || alpha: 5


In [76]:
xgb_params['alpha'] = alpha_list
xgb_params['best_iter'] = best_iter
xgb_params.head()

,n_estimators,eta,min_child_weight,max_depth,colsample_bytree,subsample,alpha,best_iter
0,100,0.01,6,3,0.8,0.9,2,363
0,100,0.01,7,3,0.8,0.8,2,262
0,100,0.01,7,3,0.8,0.9,2,207
0,100,0.01,5,3,0.8,0.9,0,334
0,100,0.01,7,3,0.8,0.8,5,281


In [ ]:
xgb_params.to_csv('hyperparameter_xgb_last_testing.csv', index=False)

In [77]:
best_iter = xgb_params['best_iter'].to_list()
best_iter[0]

363

In [62]:
train.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,holiday,sin_time,cos_time,DI,power,적정온도차이
0,1,18.6,0.0,0.9,42.0,6,2,0,0.000000,1.000000,234.154,1085.28,-7.4
1,1,18.0,0.0,1.1,45.0,6,2,0,0.258819,0.965926,219.280,1047.36,-8.0
2,1,17.7,0.0,1.5,45.0,6,2,0,0.500000,0.866025,205.672,974.88,-8.3
3,1,16.7,0.0,1.4,48.0,6,2,0,0.707107,0.707107,167.011,953.76,-9.3
4,1,18.4,0.0,2.8,43.0,6,2,0,0.866025,0.500000,229.592,986.40,-7.6


In [78]:
preds = np.array([])
for i in tqdm(range(100)):
    pred_df = pd.DataFrame()
    for seed in range(0, 6):
        x_train = train.loc[train['건물번호']==i+1,].drop(['건물번호', 'power'], axis=1)
        y_train = train.loc[train['건물번호']==i+1, 'power']
        x_test = test.loc[test['건물번호']==i+1,].drop('건물번호', axis=1)

        xgb = XGBRegressor(seed=seed, n_estimators=best_iter[i], eta=0.01,
                            min_child_weight=xgb_params.iloc[i, 2], max_depth=xgb_params.iloc[i, 3],
                            colsample_bytree=xgb_params.iloc[i, 4], subsample=xgb_params.iloc[i, 5])
        if xgb_params.iloc[i, 6] != 0:
            xgb.set_params(**{'objective':weighted_mse(xgb_params.iloc[i, 6])})

        xgb.fit(x_train[feature_list[i]], y_train)
        y_pred = xgb.predict(x_test[feature_list[i]])
        pred_df.loc[:, seed] = y_pred

    pred = pred_df.mean(axis=1)
    preds = np.append(preds, pred)

100%|██████████| 100/100 [07:02<00:00,  4.22s/it]


In [79]:
sub = pd.read_csv('sample_submission.csv')
sub['answer'] = preds
sub.to_csv('xgb_sub_26.csv', index=False)

test와 train의 분포 차이 확인  
random forest 모델 확인  
과대 예측 잡기  
54번 건물 주의(튀는 애 발견, outlier 제거 후 시도 예를 들면 뒤에서 1000개 빼서 수행)  
feature selection 요망  